In [6]:
%matplotlib inline

import numpy as np
import pandas as pd
from sklearn.datasets import fetch_openml
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
import time

import warnings
warnings.filterwarnings('ignore')

import theano 
from theano import tensor as T
from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStreams
print(theano.config.device) # We're using CPUs (for now)
print(theano.config.floatX) # Should be 64 bit for CPUs

np.random.seed(0)

cpu
float64


In [29]:
train = pd.read_csv("data/train.csv")
X = np.array(train)
Y = np.array(np.array(train["Cover_Type"].tolist()))
n = X.shape[0]

shuffle = np.random.permutation(np.arange(n))
X, Y = X[shuffle], Y[shuffle]

percent_in_dev = .2
dev_slice = int(percent_in_dev * n)

# Set some variables to hold test, dev, and training data.
dev_dat, dev_lab = X[:dev_slice], Y[:dev_slice]
train_dat, train_lab = X[dev_slice:], Y[dev_slice:]
# train_df = train.iloc[shuffle,:].iloc[dev_slice: , :]

n_dev = dev_dat.shape[0]
n_train = train_dat.shape[0]
n_feat = train_dat.shape[1]

def binarizeY(data):
    binarized_data = np.zeros((data.size,10))
    for j in range(0,data.size):
        feature = data[j:j+1]
        i = feature.astype(np.int64) 
        binarized_data[j,i]=1
    return binarized_data

train_lab_b = binarizeY(train_lab)
dev_lab_b = binarizeY(dev_labels)
n_class = train_lab_b[1].size

print(f'Dev size: {n_dev}')
print(f'Train size: {n_train}')
print(f'Features: {n_feat}')
print(f'Classes: {n_class}')

Dev size: 3024
Train size: 12096
Features: 56
Classes: 10


In [73]:
## (1) Parameters
n_hidden_nodes = 50 
w_1 = theano.shared(np.asarray((np.random.randn(*(n_feat, n_hidden_nodes))*.01)))
w_2 = theano.shared(np.asarray((np.random.randn(*(n_hidden_nodes, n_hidden_nodes))*.01)))
w_3 = theano.shared(np.asarray((np.random.randn(*(n_hidden_nodes, n_class))*.01)))
params = [w_1, w_2, w_3]


## (2) Model
X = T.matrix()
Y = T.matrix()
# Two notes:
# First, feed forward is the composition of layers (dot product + activation function)
# Second, activation on the hidden layer still uses sigmoid
def model(X, w_1, w_2, w_3):
    rectifier = lambda first_layer, second_layer : T.nnet.relu(T.dot(first_layer, second_layer))
    X_w_1 = rectifier(X, w_1)
    w_1_2 = rectifier(X_w_1, w_2)
    w_2_3 = rectifier(w_1_2, w_3)
    return T.nnet.softmax(w_2_3)
y_hat = model(X, w_1, w_2, w_3)


## (3) Cost...same as logistic regression
cost = T.mean(T.nnet.categorical_crossentropy(y_hat, Y))


## (4) Minimization.  Update rule changes to backpropagation.
alpha = 0.01
def backprop(cost, w):
    grads = T.grad(cost=cost, wrt=w)
    updates = []
    for w1, grad in zip(w, grads):
        updates.append([w1, w1 - grad * alpha])
    return updates
update = backprop(cost, params)
train = theano.function(inputs=[X, Y], outputs=cost, updates=update, allow_input_downcast=True)
y_pred = T.argmax(y_hat, axis=1)
predict = theano.function(inputs=[X], outputs=y_pred, allow_input_downcast=True)

miniBatchSize = 1
def gradientDescentStochastic(epochs):
    trainTime = 0.0
    predictTime = 0.0
    start_time = time.time()
    for i in range(epochs):
        for start, end in zip(range(0, n_train, miniBatchSize), range(miniBatchSize, n_train, miniBatchSize)):
            cost = train(train_dat[start:end], train_lab_b[start:end])
        trainTime =  trainTime + (time.time() - start_time)
        print('%d) accuracy = %.4f' %(i+1, np.mean(np.argmax(dev_lab_b, axis=1) == predict(dev_dat))))
    print('train time = %.2f' %(trainTime))

gradientDescentStochastic(10)

start_time = time.time()
predict(dev_dat)   
print('predict time = %.2f' %(time.time() - start_time))

1) accuracy = 0.0000
2) accuracy = 0.0000
3) accuracy = 0.0000
4) accuracy = 0.0000
5) accuracy = 0.0000
6) accuracy = 0.0000
7) accuracy = 0.0000
8) accuracy = 0.0000
9) accuracy = 0.0000
10) accuracy = 0.0000
train time = 67.25
predict time = 0.00
